# AI - Keras through TensorFlow


# Sommaire

[I. Understanding Keras layers](#Partie1)

[II. Using the Keras Sequential API](#Partie2)

[III. Using the Keras Functional API](#Partie3)

[IV. Using the Keras Subclassing API](#Partie4)

[V. Using the Keras Preprocessing API](#Partie5)

## I. Understanding Keras layers<a class="anchor" id="Partie1"></a>

## II. Using the Keras Sequential API<a class="anchor" id="Partie2"></a>

## III. Using the Keras Functional API<a class="anchor" id="Partie3"></a>

## IV. Using the Keras Subclassing API<a class="anchor" id="Partie4"></a>

## V. Using the Keras Preprocessing API<a class="anchor" id="Partie5"></a>